In [1]:
import csvms.engine

INFO:csvms:version:0.2.4


In [2]:
%%sql
CREATE TABLE lista_frutas (
    nm_fruta TEXT ,
    tp_fruta TEXT )


INFO:SQL:True


In [3]:
%%sql
INSERT INTO lista_frutas VALUES ('banana','doce');
INSERT INTO lista_frutas VALUES ('limão','azedo');
INSERT INTO lista_frutas VALUES ('maçã','doce');

INFO:csvms:Row inserted
INFO:SQL:True
INFO:csvms:Row inserted
INFO:SQL:True
INFO:csvms:Row inserted
INFO:SQL:True


In [4]:
%%sql
COMMIT;
 select * from lista_frutas;

INFO:SQL:True
INFO:SQL:TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|     doce|
  1|    limão|    azedo|
  2|     maçã|     doce|
   +---------+---------+



In [5]:
%%sql
UPDATE lista_frutas 
   SET tp_fruta = 'amargo'
 WHERE tp_fruta = 'doce';
 select * from lista_frutas;


INFO:csvms:Row updated
INFO:csvms:Row updated
INFO:SQL:True
INFO:SQL:TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|   amargo|
  1|    limão|    azedo|
  2|     maçã|   amargo|
   +---------+---------+



In [6]:
%%sql
ROLLBACK;
select * from lista_frutas;

INFO:SQL:True
INFO:SQL:TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|     doce|
  1|    limão|    azedo|
  2|     maçã|     doce|
   +---------+---------+



In [7]:
%%sql
drop table lista_frutas;

INFO:SQL:True


In [8]:
%%sql
select * from lista_frutas

ERROR:SQL:Table not found
